In [0]:
import numpy as np
import pandas as pd
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPool2D
from keras.layers import Reshape, Flatten, Dropout, Concatenate
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.models import Model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from sklearn.model_selection import train_test_split
import time
from keras import optimizers

Using TensorFlow backend.


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty

In [0]:
import os
os.chdir('drive')

In [0]:
#data_train = pd.read_csv('v1/Preprocessed_using_translator_train.csv')
data_train = pd.read_csv('Preprocessed_using_translator_train.csv')
data_train.shape

(400000, 3)

In [0]:
#data_test= pd.read_csv("v1/Preprocessed_using_translator_test.csv")
data_test= pd.read_csv("Preprocessed_using_translator_test.csv")
data_test.shape

(100000, 2)

In [0]:
data_train["text"] = data_train["text"].apply(str)
data_test["text"] = data_test["text"].apply(str)
ylabels = pd.get_dummies(data_train.iloc[:,2].values)

In [0]:
vocabulary_size = 2000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(data_train["text"].values)
sequences = tokenizer.texts_to_sequences(data_train["text"].values)
data = pad_sequences(sequences)

In [0]:
xtrain, xval, ytrain, yval = train_test_split(data, ylabels, test_size=0.10, random_state=960)
print(xtrain.shape, xval.shape, ytrain.shape, yval.shape)

(360000, 976) (40000, 976) (360000, 3) (40000, 3)


In [0]:
#model = Sequential()
#model.add(Conv1D(batch_size=20, input_shape=(330, 1), filters=100, kernel_size=5, strides=1, activation='relu', kernel_regularizer=regularizers.l2(0.01), use_bias=True))
#model.add(Flatten())
#model.add(MaxPooling1D(pool_size=10))
#model.add(Dropout(0.2))
#model.add(Dense(units=3, activation='softmax', use_bias=True))
#model.add(Flatten())
#myopti = optimizers.SGD(lr=0.01, momentum=0.9, decay=0.001)
#model.compile(loss='categorical_crossentropy', optimizer=myopti, metrics=['accuracy'])
#st = time.time()
#history = model.fit(datatemp, ylabels, validation_split=0.05, epochs=3, verbose=1, batch_size=20)
#en = time.time()

In [0]:
sequence_length = data.shape[1]
embedding_dim = 76
num_filters = 32
filter_sizes = [33, 33, 33]
drop = 0.2

inputs = Input(shape=(sequence_length,), dtype='int32')
embedding = Embedding(input_dim=vocabulary_size, output_dim=embedding_dim, input_length=sequence_length)(inputs)
reshape = Reshape((sequence_length,embedding_dim,1))(embedding)

c0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
c1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)
c2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal', activation='relu')(reshape)

DP0 = MaxPool2D(pool_size=(sequence_length - filter_sizes[0] + 1, 1), strides=(1,1), padding='valid')(c0)
DP1 = MaxPool2D(pool_size=(sequence_length - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(c1)
DP2 = MaxPool2D(pool_size=(sequence_length - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(c2)

CT = Concatenate(axis=1)([DP0, DP1, DP2])

flatten = Flatten()(CT)
dropout = Dropout(drop)(flatten)
output = Dense(units=3, activation='softmax')(dropout)

batch_size=32
epochs = 10

model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights.{epoch:03d}-{val_acc:.4f}.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

st = time.time()
model.fit(xtrain, ytrain, batch_size=batch_size, epochs=epochs, verbose=1, callbacks=[checkpoint], validation_data=(xval, yval))  # starts training
en = time.time()

Train on 360000 samples, validate on 40000 samples
Epoch 1/3
 20128/360000 [>.............................] - ETA: 12:23 - loss: 0.5249 - acc: 0.7531

 51456/360000 [===>..........................] - ETA: 10:56 - loss: 0.4500 - acc: 0.7965

 85472/360000 [======>.......................] - ETA: 9:39 - loss: 0.3813 - acc: 0.8303

117984/360000 [========>.....................] - ETA: 8:29 - loss: 0.3403 - acc: 0.8505

150368/360000 [===========>..................] - ETA: 7:20 - loss: 0.3141 - acc: 0.8630

...

231616/360000 [==================>...........] - ETA: 4:27 - loss: 0.1645 - acc: 0.9345

265568/360000 [=====================>........] - ETA: 3:16 - loss: 0.1643 - acc: 0.9345

298048/360000 [=======================>......] - ETA: 2:09 - loss: 0.1641 - acc: 0.9346

331680/360000 [==========================>...] - ETA: 58s - loss: 0.1639 - acc: 0.9345

360000/360000 [==============================] - 777s 2ms/step - loss: 0.1638 - acc: 0.9347 - val_loss: 0.1636 - val_acc: 0.9348

Epoch 00003: val_acc improved from 0.93037 to 0.93478, saving model to weights.003-0.9348.hdf5
